In [1]:
import os
os.chdir("../")

In [2]:
import os
import tensorflow as tf
from tensorflow.python import keras
from bert import BertModelLayer
from bert.loader import StockBertConfig, load_stock_weights

In [3]:
model_dir = "/home/naval/Desktop/Naval Projects/MicroSoft AI/bert/multi_cased_L-12_H-768_A-12"

bert_config_file = os.path.join(model_dir, "bert_config.json")
bert_ckpt_file   = os.path.join(model_dir, "bert_model.ckpt")

with tf.io.gfile.GFile(bert_config_file, "r") as reader:
  stock_params = StockBertConfig.from_json_string(reader.read())
  bert_params  = stock_params.to_bert_model_layer_params()

bert = BertModelLayer.from_params(bert_params, name="bert")

In [4]:
from tensorflow.python import keras

max_seq_len = 128
input_ids      = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="input_ids")
token_type_ids = keras.layers.Input(shape=(max_seq_len,), dtype='int32', name="token_type_ids")

output = bert([input_ids, token_type_ids])  # [batch_size, max_seq_len, hidden_size]
model = keras.Model(inputs=[input_ids, token_type_ids], outputs=output)

In [5]:
model.build(input_shape=[(None, max_seq_len),
                                 (None, max_seq_len)])

In [6]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          [(None, 128)]        0                                            
__________________________________________________________________________________________________
token_type_ids (InputLayer)     [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (BertModelLayer)           (None, 128, 768)     177262848   input_ids[0][0]                  
                                                                 token_type_ids[0][0]             
Total params: 177,262,848
Trainable params: 177,262,848
Non-trainable params: 0
__________________________________________________________________________________________________


In [7]:
output.shape

TensorShape([None, 128, 768])